In [2]:
import warnings
warnings.simplefilter('ignore')

import os, gc

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
from tqdm.auto import tqdm

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import lightgbm as lgb


In [3]:
train_info = pd.read_csv('data/A榜-训练集_分布式光伏发电预测_基本信息.csv', encoding='gb2312')
test_info = pd.read_csv('data/A榜-测试集_分布式光伏发电预测_基本信息.csv', encoding='gb2312')
train_climate = pd.read_csv('data/A榜-训练集_分布式光伏发电预测_气象变量数据.csv', encoding='gb2312')
test_climate = pd.read_csv('data/A榜-测试集_分布式光伏发电预测_气象变量数据.csv', encoding='gb2312')
train_target = pd.read_csv('data/A榜-训练集_分布式光伏发电预测_实际功率数据.csv', encoding='gb2312')
test_target = pd.read_csv('data/A榜-测试集_分布式光伏发电预测_实际功率数据.csv', encoding='gb2312')


In [4]:

train_target.columns = ['ID', 'magnifying_power', 'date_time'] + [f'p{i}' for i in range(1, 97)]
train_target['date_time'] = pd.to_datetime(train_target['date_time'])

train_data = []
for _, row in tqdm(train_target.iterrows(), total=len(train_target)):
    for i in range(1, 97):
        dt = row['date_time'] + pd.DateOffset(minutes=15*(i-1))
        train_data.append({'ID': row['ID'],
                           'magnifying_power': row['magnifying_power'],
                           'date_time': dt,
                           'target': row[f'p{i}']})
train_data = pd.DataFrame(train_data)

train_info.drop(['光伏用户名称', '经度', '纬度'], axis=1, inplace=True)
train_info.columns = ['ID', 'install_capacity']
train_data = train_data.merge(train_info, on='ID', how='left')

rename_map = {
    '光伏用户编号': 'ID', 
    '时间': 'date_time', 
    '气压(Pa）': 'atmospheric_pressure_PA', 
    '相对湿度（%）': 'moisture_percent', 
    '云量': 'cloud_cover', 
    '10米风速（10m/s）': 'wind_speed_10m/s', 
    '10米风向（°)': 'wind_direction_10m_degree',
    '温度（K）': 'temperature_K', 
    '辐照强度（J/m2）': 'irradiation_intensity_J/m2', 
    '降水（m）': 'rainfall_m', 
    '100m风速（100m/s）': 'wind_speed_100m/s', 
    '100m风向（°)': 'wind_direction_100m_degree', 
}
train_climate = train_climate.rename(columns=rename_map)
train_climate['date_time'] = pd.to_datetime(train_climate['date_time'])
train_data = train_data.merge(train_climate, on=['ID', 'date_time'], how='left')

display(train_data)



100%|██████████| 4337/4337 [00:07<00:00, 566.77it/s]


,ID,magnifying_power,date_time,target,install_capacity,atmospheric_pressure_PA,moisture_percent,cloud_cover,wind_speed_10m/s,wind_direction_10m_degree,temperature_K,irradiation_intensity_J/m2,rainfall_m,wind_speed_100m/s,wind_direction_100m_degree
0,f1,80,2022-01-03 00:00:00,0.0000,239.22,100361.6094,79.3513,0.007812,1.169500,270.0000,280.6320,0.0,0.000229,0.00000,270.0000
1,f1,80,2022-01-03 00:15:00,0.0000,239.22,100351.1476,76.8741,0.007005,1.149000,270.0000,280.6308,0.0,0.000229,0.00000,270.0000
2,f1,80,2022-01-03 00:30:00,0.0000,239.22,100341.8609,74.9771,0.006890,1.155600,270.0000,280.5719,0.0,0.000229,0.00000,270.0000
3,f1,80,2022-01-03 00:45:00,0.0000,239.22,100333.2668,73.6136,0.007236,1.180700,270.0000,280.4776,0.0,0.000229,0.00000,270.0000
4,f1,80,2022-01-03 01:00:00,0.0000,239.22,100324.8828,72.7366,0.007812,1.215700,270.0000,280.3705,0.0,0.000229,0.00000,270.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416347,f9,8000,2023-03-17 22:45:00,-0.0013,6000.00,101313.1656,82.0648,0.028876,0.000000,270.0000,289.7209,0.0,0.000016,0.17564,270.0000
416348,f9,8000,2023-03-17 23:00:00,-0.0013,6000.00,101309.0781,82.3331,0.031250,0.000000,270.0000,289.7420,0.0,0.000015,0.00000,270.0000
416349,f9,8000,2023-03-17 23:15:00,-0.0013,6000.00,101297.4523,81.7921,0.028516,0.000000,270.0000,290.0005,0.0,0.000013,0.12066,270.0000
416350,f9,8000,2023-03-17 23:30:00,-0.0013,6000.00,101280.8036,80.7190,0.022199,0.054633,270.0000,290.3679,0.0,0.000010,0.71772,270.0000


In [5]:

test_target.columns = ['ID', 'magnifying_power', 'date_time'] 
test_target['date_time'] = pd.to_datetime(test_target['date_time'])
for col in [f'p{i}' for i in range(1, 97)]:
    test_target[col] = -100

test_data = []
for _, row in tqdm(test_target.iterrows(), total=len(test_target)):
    for i in range(1, 97):
        dt = row['date_time'] + pd.DateOffset(minutes=15*(i-1))
        test_data.append({'ID': row['ID'],
                          'magnifying_power': row['magnifying_power'],
                          'date_time': dt,
                          'target': row[f'p{i}']})
test_data = pd.DataFrame(test_data)

test_info.drop(['光伏用户名称', '经度', '纬度'], axis=1, inplace=True)
test_info.columns = ['ID', 'install_capacity']
test_data = test_data.merge(test_info, on='ID', how='left')

test_climate = test_climate.rename(columns=rename_map)
test_climate['date_time'] = pd.to_datetime(test_climate['date_time'])
test_data = test_data.merge(test_climate, on=['ID', 'date_time'], how='left')

display(test_data)


100%|██████████| 825/825 [00:01<00:00, 594.91it/s]


,ID,magnifying_power,date_time,target,install_capacity,atmospheric_pressure_PA,moisture_percent,cloud_cover,wind_speed_10m/s,wind_direction_10m_degree,temperature_K,irradiation_intensity_J/m2,rainfall_m,wind_speed_100m/s,wind_direction_100m_degree
0,f1,80,2023-05-01 00:00:00,-100,239.22,99453.6250,89.8487,0.89844,0.54929,270.0000,287.6853,0.0,0.006908,0.00000,270.0000
1,f1,80,2023-05-01 00:15:00,-100,239.22,99445.4611,92.4332,0.92999,0.53548,270.0000,287.7061,0.0,0.005454,0.00000,270.0000
2,f1,80,2023-05-01 00:30:00,-100,239.22,99434.1857,92.5277,0.92273,0.59624,270.0000,287.6017,0.0,0.005404,0.00000,270.0000
3,f1,80,2023-05-01 00:45:00,-100,239.22,99420.7315,91.2865,0.90333,0.69339,270.0000,287.4312,0.0,0.006106,0.13264,270.0000
4,f1,80,2023-05-01 01:00:00,-100,239.22,99406.0313,89.8641,0.89844,0.78874,270.0000,287.2534,0.0,0.006908,0.29370,270.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79195,f9,8000,2023-07-08 22:45:00,-100,6000.00,100648.6026,31.4512,0.00000,2.91420,206.1429,300.1884,0.0,0.000000,5.85490,217.3799
79196,f9,8000,2023-07-08 23:00:00,-100,6000.00,100643.9844,31.7736,0.00000,2.80880,206.9560,300.1292,0.0,0.000008,5.72860,218.5009
79197,f9,8000,2023-07-08 23:15:00,-100,6000.00,100638.3825,33.1338,0.14679,2.72600,207.2709,300.0732,0.0,0.000027,5.60180,219.0046
79198,f9,8000,2023-07-08 23:30:00,-100,6000.00,100632.2951,35.1843,0.33322,2.66310,207.2149,300.0178,0.0,0.000053,5.47860,219.0544


In [9]:
data = pd.concat([train_data, test_data]).reset_index(drop=True)
data.shape


(495552, 15)

In [10]:

data['ID'] = data['ID'].apply(lambda x: x.replace('f', '')).astype(int)
data['day'] = data['date_time'].dt.day
data['hour'] = data['date_time'].dt.hour
data['sin_hour']= (np.pi * np.sin(data['hour']) / 12)
data['cos_hour']= (np.pi * np.cos(data['hour']) / 12)
data['date_hour'] = data['date_time'].dt.year.astype(str) + '-' + data['date_time'].dt.month.astype(str) + '-' + data['date_time'].dt.day.astype(str) +':' + data['date_time'].dt.hour.astype(str)

for col in tqdm(['atmospheric_pressure_PA',
                 'moisture_percent',
                 'cloud_cover',
                 'wind_speed_10m/s',
                 'wind_direction_10m_degree', 
                 'temperature_K',
                 'irradiation_intensity_J/m2', 
                 'rainfall_m', 
                 'wind_speed_100m/s',
                 'wind_direction_100m_degree']):
    for m in ['mean', 'std', 'skew']:
        data[f'{col}_gby_station_hour_{m}'] = data.groupby(['date_hour', 'ID'])[col].transform(m)
        
del data['date_hour']

100%|██████████| 10/10 [00:56<00:00,  5.64s/it]


In [11]:

train_data = data[data['target'] != -100].copy().reset_index(drop=True)
train_data = train_data[train_data['target'].notna()].copy().reset_index(drop=True)

test = data[data['target'] == -100].copy().reset_index(drop=True)

print(train_data.shape, test.shape)

print(train_data['date_time'].max(), train_data['date_time'].min())
display(train_data)

print(test['date_time'].max(), test['date_time'].min())
display(test)

(409828, 49) (79200, 49)
2023-04-30 23:45:00 2022-01-03 00:00:00


,ID,magnifying_power,date_time,target,install_capacity,atmospheric_pressure_PA,moisture_percent,cloud_cover,wind_speed_10m/s,wind_direction_10m_degree,temperature_K,irradiation_intensity_J/m2,rainfall_m,wind_speed_100m/s,wind_direction_100m_degree,day,hour,sin_hour,cos_hour,atmospheric_pressure_PA_gby_station_hour_mean,atmospheric_pressure_PA_gby_station_hour_std,atmospheric_pressure_PA_gby_station_hour_skew,moisture_percent_gby_station_hour_mean,moisture_percent_gby_station_hour_std,moisture_percent_gby_station_hour_skew,cloud_cover_gby_station_hour_mean,cloud_cover_gby_station_hour_std,cloud_cover_gby_station_hour_skew,wind_speed_10m/s_gby_station_hour_mean,wind_speed_10m/s_gby_station_hour_std,wind_speed_10m/s_gby_station_hour_skew,wind_direction_10m_degree_gby_station_hour_mean,wind_direction_10m_degree_gby_station_hour_std,wind_direction_10m_degree_gby_station_hour_skew,temperature_K_gby_station_hour_mean,temperature_K_gby_station_hour_std,temperature_K_gby_station_hour_skew,irradiation_intensity_J/m2_gby_station_hour_mean,irradiation_intensity_J/m2_gby_station_hour_std,irradiation_intensity_J/m2_gby_station_hour_skew,rainfall_m_gby_station_hour_mean,rainfall_m_gby_station_hour_std,rainfall_m_gby_station_hour_skew,wind_speed_100m/s_gby_station_hour_mean,wind_speed_100m/s_gby_station_hour_std,wind_speed_100m/s_gby_station_hour_skew,wind_direction_100m_degree_gby_station_hour_mean,wind_direction_100m_degree_gby_station_hour_std,wind_direction_100m_degree_gby_station_hour_skew
0,1,80,2022-01-03 00:00:00,0.0000,239.22,100361.6094,79.3513,0.007812,1.169500,270.0000,280.6320,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.000410,1.331078,1.163700,0.014193,0.326469,270.000000,0.000000,0.000000,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000e+00,0.0,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
1,1,80,2022-01-03 00:15:00,0.0000,239.22,100351.1476,76.8741,0.007005,1.149000,270.0000,280.6308,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.000410,1.331078,1.163700,0.014193,0.326469,270.000000,0.000000,0.000000,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000e+00,0.0,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
2,1,80,2022-01-03 00:30:00,0.0000,239.22,100341.8609,74.9771,0.006890,1.155600,270.0000,280.5719,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.000410,1.331078,1.163700,0.014193,0.326469,270.000000,0.000000,0.000000,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000e+00,0.0,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
3,1,80,2022-01-03 00:45:00,0.0000,239.22,100333.2668,73.6136,0.007236,1.180700,270.0000,280.4776,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.000410,1.331078,1.163700,0.014193,0.326469,270.000000,0.000000,0.000000,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000e+00,0.0,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
4,1,80,2022-01-03 01:00:00,0.0000,239.22,100324.8828,72.7366,0.007812,1.215700,270.0000,280.3705,0.0,0.000229,0.00000,270.0000,3,1,0.220297,0.141451,100311.022475,12.350572,-0.194628,72.461375,0.226355,0.491173,0.008389,0.000410,-1.331078,1.260675,0.034745,-0.734618,270.000000,0.000000,0.000000,280.261950,0.079826,1.103599,0.0,0.0,0.0,0.000229,0.000000e+00,0.0,0.104103,0.087476,-0.161915,270.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409823,9,8000,2023-03-17 22:45:00,-0.0013,6000.00,101313.1656,82.0648,0.028876,0.000000,270.0000,289.7209,0.0,0.000016,0.17564,270.0000,17,22,-0.002317,-0.261789,101309.490925,3.672262,-0.392258,80.990450,0.8

2023-07-31 23:45:00 2023-05-01 00:00:00


,ID,magnifying_power,date_time,target,install_capacity,atmospheric_pressure_PA,moisture_percent,cloud_cover,wind_speed_10m/s,wind_direction_10m_degree,temperature_K,irradiation_intensity_J/m2,rainfall_m,wind_speed_100m/s,wind_direction_100m_degree,day,hour,sin_hour,cos_hour,atmospheric_pressure_PA_gby_station_hour_mean,atmospheric_pressure_PA_gby_station_hour_std,atmospheric_pressure_PA_gby_station_hour_skew,moisture_percent_gby_station_hour_mean,moisture_percent_gby_station_hour_std,moisture_percent_gby_station_hour_skew,cloud_cover_gby_station_hour_mean,cloud_cover_gby_station_hour_std,cloud_cover_gby_station_hour_skew,wind_speed_10m/s_gby_station_hour_mean,wind_speed_10m/s_gby_station_hour_std,wind_speed_10m/s_gby_station_hour_skew,wind_direction_10m_degree_gby_station_hour_mean,wind_direction_10m_degree_gby_station_hour_std,wind_direction_10m_degree_gby_station_hour_skew,temperature_K_gby_station_hour_mean,temperature_K_gby_station_hour_std,temperature_K_gby_station_hour_skew,irradiation_intensity_J/m2_gby_station_hour_mean,irradiation_intensity_J/m2_gby_station_hour_std,irradiation_intensity_J/m2_gby_station_hour_skew,rainfall_m_gby_station_hour_mean,rainfall_m_gby_station_hour_std,rainfall_m_gby_station_hour_skew,wind_speed_100m/s_gby_station_hour_mean,wind_speed_100m/s_gby_station_hour_std,wind_speed_100m/s_gby_station_hour_skew,wind_direction_100m_degree_gby_station_hour_mean,wind_direction_100m_degree_gby_station_hour_std,wind_direction_100m_degree_gby_station_hour_skew
0,1,80,2023-05-01 00:00:00,-100.0,239.22,99453.6250,89.8487,0.89844,0.54929,270.0000,287.6853,0.0,0.006908,0.00000,270.0000,1,0,0.000000,0.261799,99438.500825,14.277667,-0.433893,91.524025,1.251279,-1.004860,0.913623,0.015136,0.105775,0.593600,0.071429,1.299549,270.00000,0.000000,0.000000,287.606075,0.125009,-1.313828,0.0,0.0,0.0,0.005968,0.000704,0.992594,0.033160,0.066320,2.000000,270.000000,0.000000,0.000000
1,1,80,2023-05-01 00:15:00,-100.0,239.22,99445.4611,92.4332,0.92999,0.53548,270.0000,287.7061,0.0,0.005454,0.00000,270.0000,1,0,0.000000,0.261799,99438.500825,14.277667,-0.433893,91.524025,1.251279,-1.004860,0.913623,0.015136,0.105775,0.593600,0.071429,1.299549,270.00000,0.000000,0.000000,287.606075,0.125009,-1.313828,0.0,0.0,0.0,0.005968,0.000704,0.992594,0.033160,0.066320,2.000000,270.000000,0.000000,0.000000
2,1,80,2023-05-01 00:30:00,-100.0,239.22,99434.1857,92.5277,0.92273,0.59624,270.0000,287.6017,0.0,0.005404,0.00000,270.0000,1,0,0.000000,0.261799,99438.500825,14.277667,-0.433893,91.524025,1.251279,-1.004860,0.913623,0.015136,0.105775,0.593600,0.071429,1.299549,270.00000,0.000000,0.000000,287.606075,0.125009,-1.313828,0.0,0.0,0.0,0.005968,0.000704,0.992594,0.033160,0.066320,2.000000,270.000000,0.000000,0.000000
3,1,80,2023-05-01 00:45:00,-100.0,239.22,99420.7315,91.2865,0.90333,0.69339,270.0000,287.4312,0.0,0.006106,0.13264,270.0000,1,0,0.000000,0.261799,99438.500825,14.277667,-0.433893,91.524025,1.251279,-1.004860,0.913623,0.015136,0.105775,0.593600,0.071429,1.299549,270.00000,0.000000,0.000000,287.606075,0.125009,-1.313828,0.0,0.0,0.0,0.005968,0.000704,0.992594,0.033160,0.066320,2.000000,270.000000,0.000000,0.000000
4,1,80,2023-05-01 01:00:00,-100.0,239.22,99406.0313,89.8641,0.89844,0.78874,270.0000,287.2534,0.0,0.006908,0.29370,270.0000,1,1,0.220297,0.141451,99384.084250,18.580239,0.103268,89.520825,0.359312,0.009150,0.946800,0.043189,0.029360,0.859895,0.052969,-0.960018,270.00000,0.000000,0.000000,287.072125,0.146039,0.378608,0.0,0.0,0.0,0.007160,0.000189,-0.993101,0.426250,0.101171,-0.763975,270.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79195,9,8000,2023-07-08 22:45:00,-100.0,6000.00,100648.6026,31.4512,0.00000,2.91420,206.1429,300.1884,0.0,0.000000,5.85490,217.3799,8,22,-0.002317,-0.261789,100647.643875,3.923477,-1.633682,32.422600,0.937234,0.410217,0.0

In [12]:

split_day = train_data['date_time'].max() - pd.DateOffset(days=70)

train = train_data[train_data['date_time'] <  split_day].reset_index(drop=True)
valid = train_data[train_data['date_time'] >= split_day].reset_index(drop=True)

display(train)
display(valid)

train.shape, valid.shape


,ID,magnifying_power,date_time,target,install_capacity,atmospheric_pressure_PA,moisture_percent,cloud_cover,wind_speed_10m/s,wind_direction_10m_degree,temperature_K,irradiation_intensity_J/m2,rainfall_m,wind_speed_100m/s,wind_direction_100m_degree,day,hour,sin_hour,cos_hour,atmospheric_pressure_PA_gby_station_hour_mean,atmospheric_pressure_PA_gby_station_hour_std,atmospheric_pressure_PA_gby_station_hour_skew,moisture_percent_gby_station_hour_mean,moisture_percent_gby_station_hour_std,moisture_percent_gby_station_hour_skew,cloud_cover_gby_station_hour_mean,cloud_cover_gby_station_hour_std,cloud_cover_gby_station_hour_skew,wind_speed_10m/s_gby_station_hour_mean,wind_speed_10m/s_gby_station_hour_std,wind_speed_10m/s_gby_station_hour_skew,wind_direction_10m_degree_gby_station_hour_mean,wind_direction_10m_degree_gby_station_hour_std,wind_direction_10m_degree_gby_station_hour_skew,temperature_K_gby_station_hour_mean,temperature_K_gby_station_hour_std,temperature_K_gby_station_hour_skew,irradiation_intensity_J/m2_gby_station_hour_mean,irradiation_intensity_J/m2_gby_station_hour_std,irradiation_intensity_J/m2_gby_station_hour_skew,rainfall_m_gby_station_hour_mean,rainfall_m_gby_station_hour_std,rainfall_m_gby_station_hour_skew,wind_speed_100m/s_gby_station_hour_mean,wind_speed_100m/s_gby_station_hour_std,wind_speed_100m/s_gby_station_hour_skew,wind_direction_100m_degree_gby_station_hour_mean,wind_direction_100m_degree_gby_station_hour_std,wind_direction_100m_degree_gby_station_hour_skew
0,1,80,2022-01-03 00:00:00,0.0000,239.22,100361.6094,79.3513,0.007812,1.1695,270.0,280.6320,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.00041,1.331078,1.163700,0.014193,0.326469,270.0,0.0,0.0,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
1,1,80,2022-01-03 00:15:00,0.0000,239.22,100351.1476,76.8741,0.007005,1.1490,270.0,280.6308,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.00041,1.331078,1.163700,0.014193,0.326469,270.0,0.0,0.0,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
2,1,80,2022-01-03 00:30:00,0.0000,239.22,100341.8609,74.9771,0.006890,1.1556,270.0,280.5719,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.00041,1.331078,1.163700,0.014193,0.326469,270.0,0.0,0.0,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
3,1,80,2022-01-03 00:45:00,0.0000,239.22,100333.2668,73.6136,0.007236,1.1807,270.0,280.4776,0.0,0.000229,0.00000,270.0000,3,0,0.000000,0.261799,100346.971175,12.188040,0.184927,76.204025,2.487970,0.530121,0.007236,0.00041,1.331078,1.163700,0.014193,0.326469,270.0,0.0,0.0,280.578075,0.072620,-1.237962,0.0,0.0,0.0,0.000229,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
4,1,80,2022-01-03 01:00:00,0.0000,239.22,100324.8828,72.7366,0.007812,1.2157,270.0,280.3705,0.0,0.000229,0.00000,270.0000,3,1,0.220297,0.141451,100311.022475,12.350572,-0.194628,72.461375,0.226355,0.491173,0.008389,0.00041,-1.331078,1.260675,0.034745,-0.734618,270.0,0.0,0.0,280.261950,0.079826,1.103599,0.0,0.0,0.0,0.000229,0.000000,0.000000,0.104103,0.087476,-0.161915,270.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349589,9,8000,2023-02-12 22:45:00,-0.0014,6000.00,101081.5841,44.9607,1.000000,0.0000,270.0,290.0925,0.0,0.008760,0.99165,191.2593,12,22,-0.002317,-0.261789,101081.456550,1.063704,-1.264125,43.840625,1.130427,-0.625887,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,270.0,0.0,0.0,289.814375,0.202400,1.

,ID,magnifying_power,date_time,target,install_capacity,atmospheric_pressure_PA,moisture_percent,cloud_cover,wind_speed_10m/s,wind_direction_10m_degree,temperature_K,irradiation_intensity_J/m2,rainfall_m,wind_speed_100m/s,wind_direction_100m_degree,day,hour,sin_hour,cos_hour,atmospheric_pressure_PA_gby_station_hour_mean,atmospheric_pressure_PA_gby_station_hour_std,atmospheric_pressure_PA_gby_station_hour_skew,moisture_percent_gby_station_hour_mean,moisture_percent_gby_station_hour_std,moisture_percent_gby_station_hour_skew,cloud_cover_gby_station_hour_mean,cloud_cover_gby_station_hour_std,cloud_cover_gby_station_hour_skew,wind_speed_10m/s_gby_station_hour_mean,wind_speed_10m/s_gby_station_hour_std,wind_speed_10m/s_gby_station_hour_skew,wind_direction_10m_degree_gby_station_hour_mean,wind_direction_10m_degree_gby_station_hour_std,wind_direction_10m_degree_gby_station_hour_skew,temperature_K_gby_station_hour_mean,temperature_K_gby_station_hour_std,temperature_K_gby_station_hour_skew,irradiation_intensity_J/m2_gby_station_hour_mean,irradiation_intensity_J/m2_gby_station_hour_std,irradiation_intensity_J/m2_gby_station_hour_skew,rainfall_m_gby_station_hour_mean,rainfall_m_gby_station_hour_std,rainfall_m_gby_station_hour_skew,wind_speed_100m/s_gby_station_hour_mean,wind_speed_100m/s_gby_station_hour_std,wind_speed_100m/s_gby_station_hour_skew,wind_direction_100m_degree_gby_station_hour_mean,wind_direction_100m_degree_gby_station_hour_std,wind_direction_100m_degree_gby_station_hour_skew
0,1,80,2023-02-19 23:45:00,-0.0012,239.22,100115.2707,77.8871,0.999490,0.000000,270.0000,284.1666,0.0,0.000020,0.00000,270.0000,19,23,-0.221540,-0.139495,100121.579150,5.831661,0.301319,80.374825,2.221448,0.170976,0.999677,0.000243,0.953180,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000,284.397725,0.190839,-0.275384,0.0,0.0,0.0,0.000057,3.126621e-05,-0.231553,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
1,1,80,2023-02-20 00:00:00,-0.0011,239.22,100114.3750,76.8372,1.000000,0.000000,270.0000,284.0205,0.0,0.000000,0.00000,270.0000,20,0,0.000000,0.261799,100119.072075,4.427235,0.303743,76.725300,0.319972,0.493931,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000,283.867475,0.128899,0.074593,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
2,1,80,2023-02-20 00:15:00,-0.0011,239.22,100116.7344,76.4058,1.000000,0.000000,270.0000,283.9112,0.0,0.000000,0.00000,270.0000,20,0,0.000000,0.261799,100119.072075,4.427235,0.303743,76.725300,0.319972,0.493931,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000,283.867475,0.128899,0.074593,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
3,1,80,2023-02-20 00:30:00,-0.0011,239.22,100120.7825,76.5367,1.000000,0.000000,270.0000,283.8199,0.0,0.000000,0.00000,270.0000,20,0,0.000000,0.261799,100119.072075,4.427235,0.303743,76.725300,0.319972,0.493931,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000,283.867475,0.128899,0.074593,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
4,1,80,2023-02-20 00:45:00,-0.0012,239.22,100124.3964,77.1215,1.000000,0.000000,270.0000,283.7183,0.0,0.000000,0.00000,270.0000,20,0,0.000000,0.261799,100119.072075,4.427235,0.303743,76.725300,0.319972,0.493931,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000,283.867475,0.128899,0.074593,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,270.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60229,9,8000,2023-03-17 22:45:00,-0.0013,6000.00,101313.1656,82.0648,0.028876,0.000000,270.0000,289.7209,0.0,0.000016,0.17564,270.0000,17,22,-0.002317,-0.261789,101309.490925,3.6722

((349594, 49), (60234, 49))

In [13]:
feats = [f for f in train.columns if f not in ['date_time', 'target']]
len(feats)

47

In [14]:
Xtr = train[feats].values
Ytr = train['target'].values
Xvl = valid[feats].values
Yvl = valid['target'].values

params = {
    'num_iterations': 2000,
    'verbose': -1,
    'objective': 'rmse',
    'num_leaves': 64,
    'learning_rate': 0.01,
    'colsample_bytree' : 0.8,
    'colsample_bynode' : 0.8,
    'reg_alpha': 0.2,
    'reg_lambda': 0.3,
    'min_data_in_leaf': 32,
    'max_depth': 6,
    "n_jobs": 4
}

lgbm = lgb.LGBMRegressor(**params, random_state=42)
lgbm = lgb.train(params, 
                 lgb.Dataset(Xtr, Ytr),
                 valid_sets=[lgb.Dataset(Xvl, Yvl)],
                 callbacks=[lgb.early_stopping(stopping_rounds=100)])

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[566]	valid_0's l2: 0.0386343


In [15]:
df_importance = pd.DataFrame({
    'column': feats,
    'importance': lgbm.feature_importance(),
})
display(df_importance.sort_values('importance', ascending=False).reset_index(drop=True))

,column,importance
0,hour,2823
1,irradiation_intensity_J/m2,2713
2,irradiation_intensity_J/m2_gby_station_hour_mean,1797
3,atmospheric_pressure_PA_gby_station_hour_mean,1675
4,moisture_percent_gby_station_hour_mean,1461
5,atmospheric_pressure_PA,1439
6,irradiation_intensity_J/m2_gby_station_hour_skew,1401
7,ID,1342
8,install_capacity,1233
9,irradiation_intensity_J/m2_gby_station_hour_std,1180


In [16]:
pred = lgbm.predict(Xvl)
score = 1 / (1 + mean_squared_error(valid['target'], pred, squared=False))
score

0.8357318748628274

In [17]:
Xtr = train_data[feats].values
Ytr = train_data['target'].values
params['num_iterations'] = lgbm.best_iteration
lgbm = lgb.LGBMRegressor(**params, random_state=42)
lgbm = lgb.train(params, lgb.Dataset(Xtr, Ytr))
pred = lgbm.predict(test[feats])

In [18]:
test['pred'] = pred
df = test[['ID', 'magnifying_power', 'date_time', 'pred']]
df['day'] = df['date_time'].astype(str).apply(lambda x: x.split()[0])
df = df.groupby(['ID', 'day'])['pred'].agg(list).to_frame(name='pred').reset_index()

def format_datetime(x):
    year, month, day = x.split('-')
    if month.startswith('0'):
        month = month.replace('0', '')
    if day.startswith('0'):
        day = day.replace('0', '')
    return '-'.join([year, month, day]) + ' 0:00'

df['时间'] = df['day'].apply(format_datetime)
df['光伏用户编号'] = df['ID'].apply(lambda x: f'f{x}')

In [19]:
subm_data = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    di = {'光伏用户编号': row["光伏用户编号"], '时间': row['时间']}
    for i, p in enumerate(row['pred']):
        di[f'p{i+1}'] = p
    subm_data.append(di)
subm_data = pd.DataFrame(subm_data)
sub = pd.read_csv('data/submit_example.csv')
sub = sub[['光伏用户编号', '综合倍率', '时间']].merge(subm_data, on=['光伏用户编号', '时间'], how='left')
display(sub)

100%|██████████| 825/825 [00:00<00:00, 20677.65it/s]


,光伏用户编号,综合倍率,时间,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,p26,p27,p28,p29,p30,p31,p32,p33,p34,p35,p36,p37,p38,p39,p40,p41,p42,p43,p44,p45,p46,p47,p48,p49,p50,p51,p52,p53,p54,p55,p56,p57,p58,p59,p60,p61,p62,p63,p64,p65,p66,p67,p68,p69,p70,p71,p72,p73,p74,p75,p76,p77,p78,p79,p80,p81,p82,p83,p84,p85,p86,p87,p88,p89,p90,p91,p92,p93,p94,p95,p96
0,f1,80,2023-5-1 0:00,0.002519,0.002519,0.002519,0.002310,0.001344,0.001537,0.001558,0.001415,0.001682,0.001682,0.001505,0.001485,0.000901,0.000884,0.000884,0.000853,0.001487,0.001487,0.001487,0.001487,0.002461,0.002350,0.002291,0.002291,0.156120,0.156645,0.157539,0.159672,0.412825,0.493053,0.584445,0.660085,0.889227,0.960190,1.031505,1.070231,1.234810,1.291024,1.368094,1.391651,1.494104,1.538339,1.555966,1.577415,1.659796,1.750693,1.758843,1.762217,1.730022,1.737141,1.738311,1.738311,1.703205,1.703205,1.704675,1.696084,1.522489,1.458870,1.416016,1.381615,1.015188,0.920628,0.808534,0.741105,0.500430,0.415538,0.316020,0.251618,0.168799,0.129826,0.086321,0.049652,0.033701,0.001979,-0.008182,-0.009453,0.000203,0.000257,-0.001327,-0.001293,-0.003241,-0.003275,-0.003246,-0.003246,-0.002539,-0.002640,-0.002640,-0.002640,-0.000365,-0.000336,-0.000380,-0.000380,-0.001688,-0.001688,-0.001764,-0.001852
1,f1,80,2023-5-2 0:00,0.031429,0.031056,0.030240,0.030199,0.000117,0.000218,0.000409,0.000528,0.000403,0.000791,0.000791,0.000403,0.000324,0.000324,0.000324,0.000713,0.001326,0.001326,0.001326,0.001503,0.002408,0.002408,0.002472,0.002616,0.153400,0.154479,0.155220,0.157702,0.390188,0.436516,0.478232,0.495198,0.542421,0.552009,0.579419,0.619028,0.635374,0.674883,0.683041,0.726249,0.655853,0.686129,0.686129,0.686129,0.683277,0.683277,0.713846,0.743275,0.630862,0.630230,0.611337,0.605605,0.521610,0.521610,0.521610,0.528647,0.396938,0.396081,0.380458,0.346024,0.246786,0.239477,0.233737,0.231652,0.088803,0.084655,0.080444,0.075848,0.019057,0.015888,0.007221,0.007815,0.000850,-0.002069,-0.003383,-0.003035,0.000704,0.000704,0.000704,0.000704,-0.000381,-0.000381,-0.000378,-0.000378,-0.000098,-0.000098,-0.000098,-0.000098,0.001167,0.001290,0.001390,0.001530,0.003585,0.003216,0.003081,0.002921
2,f1,80,2023-5-3 0:00,0.006198,0.006198,0.006198,0.006198,0.003099,0.003099,0.003099,0.003130,0.003286,0.003286,0.003286,0.003252,0.003624,0.003624,0.003624,0.003624,0.004569,0.004569,0.004569,0.004569,0.005466,0.005466,0.005466,0.005466,0.113530,0.113530,0.113530,0.113903,0.387608,0.462709,0.528564,0.604506,0.776842,0.833806,0.917107,0.976576,1.098281,1.132613,1.154037,1.171854,1.321727,1.384026,1.460967,1.524313,1.468208,1.423422,1.253620,1.137880,1.121601,1.132163,1.190257,1.290984,1.326081,1.343351,1.342237,1.309935,1.058038,1.048480,1.005601,0.997897,0.620965,0.566841,0.488385,0.436918,0.316663,0.254254,0.213637,0.196026,0.092099,0.084509,0.056440,0.038107,0.008446,-0.004938,-0.009091,-0.007909,0.005358,0.005358,0.005358,0.005358,0.001306,0.001120,0.000877,0.000793,0.000455,0.000455,0.000455,0.000455,0.000129,0.000128,0.000128,0.000445,0.002260,0.002260,0.002251,0.002553
3,f1,80,2023-5-4 0:00,0.010812,0.010812,0.010936,0.010936,0.004290,0.004289,0.004287,0.004287,0.002880,0.002880,0.002880,0.002880,0.003119,0.002716,0.003018,0.002972,0.004635,0.004635,0.004395,0.004395,0.005369,0.005369,0.005369,0.005471,0.177195,0.179146,0.179146,0.179146,0.510637,0.629021,0.700157,0.789461,0.974988,1.004543,1.034117,1.099244,1.182310,1.230088,1.274595,1.331913,1.413054,1.435981,1.401262,1.381032,1.387051,1.438020,1.454593,1.484195,1.165089,1.128553,1.065611,0.974517,0.923585,0.926491,0.947586,0.956437,0.762901,0.787167,0.797413,0.802704,0.607900,0.559488,0.502688,0.466151,0.285826,0.243806,0.207575,0.168572,0.088383,0.079256,0.052706,0.032850,0.009544,-0.009774,-0.013812,-0.011884,0.004456,0.004456,0.004456,0.004456,0.000906,0.000906,0.000906,0.000901,0.000826,0.000826,0.000826,0.000827,0.000279,0.000279,0.000279,0.000279,0.002987,0.002987,0.002982,0.002982
4,f1,80,2023-5-5 0:00,0.0083

In [20]:
sub.to_csv(f'lgb_base_{score}.csv', index=False)